In [65]:
import torch
import numpy as np
from models import BidirectionalLSTM

import pickle
import spacy


In [66]:
with open("vocab.pickle", "rb") as vocabf:
    vocab = pickle.load(vocabf)

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [68]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
model = BidirectionalLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                          N_LAYERS, BIDIRECTIONAL, DROPOUT).to(device)

RuntimeError: CUDA error: unspecified launch failure

In [ ]:
model.load_state_dict(torch.load("Bidirectional.pth"))

In [ ]:
d = model.state_dict()
d

In [ ]:
def sentence_to_tensor(sentence, vocab):
    nlp = spacy.load("en")
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [vocab[t] for t in tokenized]

    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    return tensor

In [ ]:
input = sentence_to_tensor("This film is great", vocab)

In [ ]:
weights = {}
gate_ids = set(zip(range(4), ["i", "f", "g", "o"]))
w_tensor_ids = ["i", "h"]

for l in range(N_LAYERS):
    for w_tensor_id in w_tensor_ids:
        w_tensor = f"rnn.weight_{w_tensor_id}h_l{l}"
        for idx, id in gate_ids:
            weights[f"W{w_tensor_id}{id}"] = d[w_tensor][(idx*HIDDEN_DIM):((idx+1)*HIDDEN_DIM)]

    

In [ ]:
 d["embedding.weight"].shape

In [ ]:
embedded = np.array([])
for i in input:
    np.append(embedded,  d["embedding.weight"][input])

In [ ]:
input